<a href="https://colab.research.google.com/github/AzucenaMV/top2000-dashboard/blob/main/top_2000_spotify_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = 'drive/MyDrive/JADS/DataVizProject/Code/'
file_top = "top2000.csv"
file_features = 'song_features_v2.csv'
file_id = 'song_id.csv'
file_artist = 'artist_features.csv'

In [ ]:
df = pd.read_csv(os.path.join(path,file_top))

In [ ]:
def clean_name(df, new_col = 'song_clean', old_col = 'song'):
  df.dropna(subset = [old_col], inplace = True)
  df[new_col] = df[old_col].str.lower()
  df[new_col] = df[new_col].str.lstrip()
  df[new_col] = df[new_col].str.replace("'","")
  df[new_col] = df[new_col].str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')
  return df

In [ ]:
df = clean_name(df)
df = clean_name(df, new_col = 'artist_clean', old_col = 'artist')

In [ ]:
# manually cleaning song names
df.loc[232,'song_clean'] = 'kronenburg park'
df.loc[2278,'song_clean'] = "rainy day woman 12"
df.loc[2817,'song_clean'] = 'everlong acoustic'
df.loc[3456,'song_clean'] = 'abergavernny'
df.loc[3482,'song_clean'] = "dont you write her off"
df.loc[3614,'song_clean'] = 'ein bisschen frieden'
df.loc[4205,'song_clean'] = 'everybody knows'

In [ ]:
TOKEN = ""

In [ ]:
# getting song id by name of the song (and artist)
data_list = []

for ind, (artist, song) in enumerate(zip(df['artist_clean'],df['song_clean'])):
  r = requests.get(f'https://api.spotify.com/v1/search?q=track:{song}%20artist:{artist}&type=track&limit=1', headers={'Authorization': f'Bearer {TOKEN}'})
  try: 
    json = r.json()['tracks']['items'][0]
    data_list.append([ind, json['id'],json['name'],json['artists'][0]['name'],json['artists'][0]['id'],json['album']['name'],json['album']['release_date'], json['popularity'],json['duration_ms'],1])
  except:
    r = requests.get(f'https://api.spotify.com/v1/search?q=track:{song}&type=track&limit=1', headers={'Authorization': f'Bearer {TOKEN}'})
    
    try:
      json = r.json()['tracks']['items'][0]
      data_list.append([ind, json['id'],json['name'],json['artists'][0]['name'],json['artists'][0]['id'],json['album']['name'],json['album']['release_date'], json['popularity'],json['duration_ms'],0])
    except:
      print(ind)
      data_list.append([ind] + [''] * 9)

1386
1918
2320
2501
3358
3456
3660
4320
4528
4529


In [ ]:
df_id = pd.DataFrame (data_list, columns = ['index', 'song_id','song_name','artist_name','artist_id','album_name','album_date','song_popularity','duration_ms','search_includes_artist'])

In [ ]:
# saving file
df_id.to_csv(os.path.join(path,file_id), index = False)

In [ ]:
df_id = pd.read_csv(os.path.join(path,file_id))

In [ ]:
TOKEN = ""

In [ ]:
# getting audio features with song id
import time

features_list = []
song_ids = df_id[df_id.song_id.notna()].song_id

for id in song_ids:
  if id != '':
    time.sleep(.5)
    x = requests.get(f'https://api.spotify.com/v1/audio-features/{id}', 
                    headers={'Authorization': f'Bearer {TOKEN}'})
    features_list.append(x.json())

In [ ]:
df_features = pd.DataFrame(features_list)

In [ ]:
df_features.shape

(4627, 18)

In [ ]:
# saving file
df_features.dropna(subset = ['id'], inplace = True)
df_features.to_csv(os.path.join(path,file_features), index = False)

In [ ]:
TOKEN = ""

In [ ]:
import time
artist_list = []
artist_ids = df_id.artist_id.unique()

for id in artist_ids:
  if id != '':
    time.sleep(.5)
    x = requests.get(f'https://api.spotify.com/v1/artists/{id}', 
                    headers={'Authorization': f'Bearer {TOKEN}'})
    response = x.json()
    artist_list.append([id,response['genres'],response['popularity'],response['name']])



In [ ]:
df_artist = pd.DataFrame (artist_list, columns = ['artist_id', 'artist_genre','artist_popularity','artist_name'])

In [ ]:
# saving artist features file
df_artist.to_csv(os.path.join(path,file_artist), index = False)

In [ ]:
df_id = pd.read_csv(os.path.join(path,file_id))
df_features = pd.read_csv(os.path.join(path,file_features))
df_artist = pd.read_csv(os.path.join(path,file_artist))

In [ ]:
# Removing possible duplicates
df_id = df_id.drop_duplicates(subset = ['song_id'], keep = 'first')
df_features = df_features.drop_duplicates(subset = ['id'], keep = 'first')

In [ ]:
# Merging all dfs
df_merged = df_id.merge(df_features, how = 'left', left_on = 'song_id', right_on = 'id', suffixes = ("","_feature"))
df_final = df_merged.merge(df_artist, how = 'left', on = 'artist_id', suffixes = ("","_artist"))

In [ ]:
df_final.shape

(4592, 31)

In [1]:
# Saving final df
file_spotify = "spotify_features.csv"
df_final.to_csv(os.path.join(path,file_spotify), index = False)